#### 여기서는 Tutorial에서 배운 개념을 이용하여 간단하게 Network parameter를 분산 환경에서 서로 복사해오는 작업을 해보겠습니다. <br>
    1. 각 actor는 network을 가지고 있다. 
    2. 학습을 하면서 일정 간격으로 actor는 learner의 파라미터를 복제해온다.

In [1]:
import ray 
import time 
import numpy as np 

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
ray.init() 

2021-01-23 07:44:53,327	INFO services.py:1173 -- View the Ray dashboard at http://127.0.0.1:8265


{'node_ip_address': '192.168.0.61',
 'raylet_ip_address': '192.168.0.61',
 'redis_address': '192.168.0.61:6379',
 'object_store_address': '/tmp/ray/session_2021-01-23_07-44-52_876201_94768/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2021-01-23_07-44-52_876201_94768/sockets/raylet',
 'webui_url': '127.0.0.1:8265',
 'session_dir': '/tmp/ray/session_2021-01-23_07-44-52_876201_94768',
 'metrics_export_port': 37420,
 'node_id': 'be3414a09c77cbfb36d74520d7524280deff15dd'}

In [3]:
class QNetwork(nn.Module):
    def __init__(self, state_size, action_size, hidden=32):
        super(QNetwork, self).__init__()

        state_size = state_size[0]
        self.fc1 = nn.Linear(state_size, hidden)
        self.fc2 = nn.Linear(hidden, action_size)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

state_size = (4, ) 
action_size = 2 
temp_net = QNetwork(state_size, action_size, 32) 
test = torch.randn(size=(4,)) 
temp_net(test), temp_net(test).shape 

(tensor([ 0.0674, -0.5022], grad_fn=<AddBackward0>), torch.Size([2]))

In [4]:
# actor의 역할은 각각 env에서 경험한 것을 buffer에 넘겨주는 역할을 합니다.
@ray.remote
class Actor:
    def __init__(self, 
                 learner: ("class: Learner class"),
                 actor_idx: ('int: Actor index'),
                 hidden: ("int: Update frequency of learner's q_behave network"), 
                 device: ("int: Cuda device number")):

        self.learner = learner # ray를 통해 공유하는 learner class입니다.
        self.device = device
        self.actor_idx = actor_idx

        # Network parameters
        self.state_dim = (16, )
        self.action_dim = 3
        self.q_behave = QNetwork(self.state_dim, self.action_dim, hidden).to(self.device)

    def explore(self):
        print("exploration start..")
        while 1:
            time.sleep(3)
            self.get_weights()
            print('updates done.')

    def get_weights(self):
        weight_copy = self.learner.return_weights.remote()
        weight_copy = ray.get(weight_copy)
        print(type(weight_copy), self.actor_idx)
        self.q_behave.load_state_dict(weight_copy)

In [5]:
# 공유 Buffer를 통해 학습을 진행하는 Learner를 정의합니다. 
# Learner는 buffer에 있는 샘플을 이용하여 network parameter를 업데이트를 하며, agent에게 network weight을 전달합니다.

@ray.remote
class Learner:
    def __init__(self, 
                 hidden: ("int: Update frequency of learner's q_behave network"), 
                 device: ("int: Cuda device number")):
        
        self.state_dim = (16, )
        self.action_dim = 3
        self.device = device
        
        self.q_behave = QNetwork(self.state_dim, self.action_dim, hidden).to(self.device)
        self.q_target = QNetwork(self.state_dim, self.action_dim, hidden).to(self.device)
        self.q_target.load_state_dict(self.q_behave.state_dict())
        self.q_target.eval()

    def return_weights(self):
        return self.q_target.state_dict() 

In [6]:
hidden = 32
device = "cuda:1" if torch.cuda.is_available() else "cpu"
# device = "cpu"

learner = Learner.remote(hidden, device) 

(pid=94899) THCudaCheck FAIL file=/opt/conda/conda-bld/pytorch_1579040055865/work/aten/src/THC/THCGeneral.cpp line=50 error=38 : no CUDA-capable device is detected
2021-01-23 07:45:00,845	ERROR worker.py:980 -- Possible unhandled error from worker: ray::Learner.__init__() (pid=94899, ip=192.168.0.61)
  File "python/ray/_raylet.pyx", line 463, in ray._raylet.execute_task
  File "python/ray/_raylet.pyx", line 415, in ray._raylet.execute_task.function_executor
  File "<ipython-input-5-7e189b24558d>", line 14, in __init__
  File "/home/sonic/.conda/envs/RL_Env/lib/python3.7/site-packages/torch/nn/modules/module.py", line 425, in to
    return self._apply(convert)
  File "/home/sonic/.conda/envs/RL_Env/lib/python3.7/site-packages/torch/nn/modules/module.py", line 201, in _apply
    module._apply(fn)
  File "/home/sonic/.conda/envs/RL_Env/lib/python3.7/site-packages/torch/nn/modules/module.py", line 223, in _apply
    param_applied = fn(param)
  File "/home/sonic/.conda/envs/RL_Env/lib/pytho

In [ ]:
learner

In [7]:
num_actors = 16 # actor의 개수

# num_actors 개수만큼 선언하고, explore 실행. actor라는 변수가 계속 중복이 되지만 실행은 잘 된다.
for actor_idx in range(num_actors):
    actor = Actor.remote(learner, actor_idx, hidden, device)
    actor.explore.remote()

(pid=94269) THCudaCheck FAIL file=/opt/conda/conda-bld/pytorch_1579040055865/work/aten/src/THC/THCGeneral.cpp line=50 error=38 : no CUDA-capable device is detected
(pid=94272) THCudaCheck FAIL file=/opt/conda/conda-bld/pytorch_1579040055865/work/aten/src/THC/THCGeneral.cpp line=50 error=38 : no CUDA-capable device is detected
(pid=94297) THCudaCheck FAIL file=/opt/conda/conda-bld/pytorch_1579040055865/work/aten/src/THC/THCGeneral.cpp line=50 error=38 : no CUDA-capable device is detected
(pid=94252) THCudaCheck FAIL file=/opt/conda/conda-bld/pytorch_1579040055865/work/aten/src/THC/THCGeneral.cpp line=50 error=38 : no CUDA-capable device is detected
(pid=94295) THCudaCheck FAIL file=/opt/conda/conda-bld/pytorch_1579040055865/work/aten/src/THC/THCGeneral.cpp line=50 error=38 : no CUDA-capable device is detected
(pid=94234) THCudaCheck FAIL file=/opt/conda/conda-bld/pytorch_1579040055865/work/aten/src/THC/THCGeneral.cpp line=50 error=38 : no CUDA-capable device is detected
(pid=94298) THCu

2021-01-23 07:44:09,708	ERROR worker.py:980 -- Possible unhandled error from worker: ray::Actor.explore() (pid=94272, ip=192.168.0.61)
  File "python/ray/_raylet.pyx", line 422, in ray._raylet.execute_task
  File "python/ray/_raylet.pyx", line 456, in ray._raylet.execute_task
  File "python/ray/_raylet.pyx", line 459, in ray._raylet.execute_task
  File "python/ray/_raylet.pyx", line 463, in ray._raylet.execute_task
  File "python/ray/_raylet.pyx", line 415, in ray._raylet.execute_task.function_executor
  File "<ipython-input-4-bb353c57f130>", line 17, in __init__
  File "/home/sonic/.conda/envs/RL_Env/lib/python3.7/site-packages/torch/nn/modules/module.py", line 425, in to
    return self._apply(convert)
  File "/home/sonic/.conda/envs/RL_Env/lib/python3.7/site-packages/torch/nn/modules/module.py", line 201, in _apply
    module._apply(fn)
  File "/home/sonic/.conda/envs/RL_Env/lib/python3.7/site-packages/torch/nn/modules/module.py", line 223, in _apply
    param_applied = fn(param)
  

2021-01-23 07:44:09,719	ERROR worker.py:980 -- Possible unhandled error from worker: ray::Actor.explore() (pid=94295, ip=192.168.0.61)
  File "python/ray/_raylet.pyx", line 422, in ray._raylet.execute_task
  File "python/ray/_raylet.pyx", line 456, in ray._raylet.execute_task
  File "python/ray/_raylet.pyx", line 459, in ray._raylet.execute_task
  File "python/ray/_raylet.pyx", line 463, in ray._raylet.execute_task
  File "python/ray/_raylet.pyx", line 415, in ray._raylet.execute_task.function_executor
  File "<ipython-input-4-bb353c57f130>", line 17, in __init__
  File "/home/sonic/.conda/envs/RL_Env/lib/python3.7/site-packages/torch/nn/modules/module.py", line 425, in to
    return self._apply(convert)
  File "/home/sonic/.conda/envs/RL_Env/lib/python3.7/site-packages/torch/nn/modules/module.py", line 201, in _apply
    module._apply(fn)
  File "/home/sonic/.conda/envs/RL_Env/lib/python3.7/site-packages/torch/nn/modules/module.py", line 223, in _apply
    param_applied = fn(param)
  

2021-01-23 07:44:09,729	ERROR worker.py:980 -- Possible unhandled error from worker: ray::Actor.explore() (pid=94220, ip=192.168.0.61)
  File "python/ray/_raylet.pyx", line 422, in ray._raylet.execute_task
  File "python/ray/_raylet.pyx", line 456, in ray._raylet.execute_task
  File "python/ray/_raylet.pyx", line 459, in ray._raylet.execute_task
  File "python/ray/_raylet.pyx", line 463, in ray._raylet.execute_task
  File "python/ray/_raylet.pyx", line 415, in ray._raylet.execute_task.function_executor
  File "<ipython-input-4-bb353c57f130>", line 17, in __init__
  File "/home/sonic/.conda/envs/RL_Env/lib/python3.7/site-packages/torch/nn/modules/module.py", line 425, in to
    return self._apply(convert)
  File "/home/sonic/.conda/envs/RL_Env/lib/python3.7/site-packages/torch/nn/modules/module.py", line 201, in _apply
    module._apply(fn)
  File "/home/sonic/.conda/envs/RL_Env/lib/python3.7/site-packages/torch/nn/modules/module.py", line 223, in _apply
    param_applied = fn(param)
  

2021-01-23 07:44:09,740	ERROR worker.py:980 -- Possible unhandled error from worker: ray::Actor.explore() (pid=94263, ip=192.168.0.61)
  File "python/ray/_raylet.pyx", line 422, in ray._raylet.execute_task
  File "python/ray/_raylet.pyx", line 456, in ray._raylet.execute_task
  File "python/ray/_raylet.pyx", line 459, in ray._raylet.execute_task
  File "python/ray/_raylet.pyx", line 463, in ray._raylet.execute_task
  File "python/ray/_raylet.pyx", line 415, in ray._raylet.execute_task.function_executor
  File "<ipython-input-4-bb353c57f130>", line 17, in __init__
  File "/home/sonic/.conda/envs/RL_Env/lib/python3.7/site-packages/torch/nn/modules/module.py", line 425, in to
    return self._apply(convert)
  File "/home/sonic/.conda/envs/RL_Env/lib/python3.7/site-packages/torch/nn/modules/module.py", line 201, in _apply
    module._apply(fn)
  File "/home/sonic/.conda/envs/RL_Env/lib/python3.7/site-packages/torch/nn/modules/module.py", line 223, in _apply
    param_applied = fn(param)
  

2021-01-23 07:44:09,750	ERROR worker.py:980 -- Possible unhandled error from worker: ray::Actor.__init__() (pid=94230, ip=192.168.0.61)
  File "python/ray/_raylet.pyx", line 463, in ray._raylet.execute_task
  File "python/ray/_raylet.pyx", line 415, in ray._raylet.execute_task.function_executor
  File "<ipython-input-4-bb353c57f130>", line 17, in __init__
  File "/home/sonic/.conda/envs/RL_Env/lib/python3.7/site-packages/torch/nn/modules/module.py", line 425, in to
    return self._apply(convert)
  File "/home/sonic/.conda/envs/RL_Env/lib/python3.7/site-packages/torch/nn/modules/module.py", line 201, in _apply
    module._apply(fn)
  File "/home/sonic/.conda/envs/RL_Env/lib/python3.7/site-packages/torch/nn/modules/module.py", line 223, in _apply
    param_applied = fn(param)
  File "/home/sonic/.conda/envs/RL_Env/lib/python3.7/site-packages/torch/nn/modules/module.py", line 423, in convert
    return t.to(device, dtype if t.is_floating_point() else None, non_blocking)
  File "/home/son